In [1]:
import os
import pathlib
import argparse
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics.functional as tF
import pytorch_lightning as pl
import tokenizers
import datasets


DEBUG_RUN = False

In [2]:
class HFDataset(torch.utils.data.Dataset):
    def __init__(self, hfdf):
        self.hfdf = hfdf

    def __getitem__(self, idx):
        return self.hfdf[idx]

    def __len__(self):
        return len(self.hfdf)

In [5]:
class LitSegmenterBaseline(pl.LightningModule):
    def __init__(
        self,
        hidden_size: int,
        tokenizer_uri: str,
        dataset_uri: str,
        batch_size: int,
        num_layers: int = 1,
        bidirectional: bool = True,
        num_classes: int = 4,
        pad_token: str = "[PAD]",
    ):
        super(LitSegmenterBaseline, self).__init__()

        self.tokenizer = tokenizers.Tokenizer.from_file(tokenizer_uri)

        self.batch_size = batch_size
        self.pad_id = self.tokenizer.get_vocab().get(pad_token, 0)

        def fn_pad_sequences(batch):
            X = [torch.tensor(x_i["input_ids"], dtype=torch.int) for x_i in batch]
            y = [torch.tensor(y_i["labels"]) for y_i in batch]

            X = nn.utils.rnn.pad_sequence(X, padding_value=self.pad_id, batch_first=True)
            y = nn.utils.rnn.pad_sequence(y, padding_value=-100, batch_first=True)

            return X, y

        self.fn_pad_sequences = fn_pad_sequences

        self.hfdf = datasets.load_from_disk(dataset_uri)

        self.embeddings = nn.Embedding(
            num_embeddings=self.tokenizer.get_vocab_size(),
            embedding_dim=768,
            padding_idx=self.pad_id,
        )

        self.lstm = nn.LSTM(
            input_size=768,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.0 if num_layers == 1 else 0.1,
            bidirectional=bidirectional,
            proj_size=0,
        )

        self.lin_out = nn.Linear(
            (1 + int(bidirectional)) * hidden_size,
            num_classes,
        )

    def forward(self, X):
        out = X

        if isinstance(out, str):
            out = self.tokenizer(out, return_tensors="pt")
            out = out["input_ids"]

        out = self.embeddings(out)
        out, *_ = self.lstm(out)
        out = self.lin_out(out)

        return out

    @staticmethod
    def _compute_pred_metrics(y_preds, y, phase: str) -> dict[str, float]:
        y_preds = y_preds.view(-1, y_preds.shape[-1])
        y = y.view(-1).squeeze()

        loss = F.cross_entropy(input=y_preds, target=y, ignore_index=-100)

        non_pad_inds = [i for i, cls_i in enumerate(y) if cls_i != -100]

        per_cls_recall = tF.recall(
            preds=y_preds[non_pad_inds, ...],
            target=y[non_pad_inds],
            num_classes=4,
            average=None,
        )

        per_cls_precision = tF.precision(
            preds=y_preds[non_pad_inds, ...],
            target=y[non_pad_inds],
            num_classes=4,
            average=None,
        )

        macro_precision = float(per_cls_precision.mean().item())
        macro_recall = float(per_cls_recall.mean().item())
        macro_f1_score = (
            2.0 * macro_precision * macro_recall / (1e-8 + macro_precision + macro_recall)
        )

        out = {
            f"{(phase + '_') if phase != 'train' else ''}loss": loss,
            **{f"{phase}_cls_{i}_precision": float(val) for i, val in enumerate(per_cls_precision)},
            **{f"{phase}_cls_{i}_recall": float(val) for i, val in enumerate(per_cls_recall)},
            f"{phase}_macro_precision": macro_precision,
            f"{phase}_macro_recall": macro_recall,
            f"{phase}_macro_f1_score": macro_f1_score,
        }

        return out

    @staticmethod
    def _agg_stats(step_outputs):
        out = {}
        agg_items = collections.defaultdict(list)

        for items in step_outputs:
            for key, val in items.items():
                if not isinstance(val, torch.Tensor):
                    val = torch.tensor(val)

                agg_items[key].append(val)

        for key, vals in agg_items.items():
            avg_vals = float(torch.stack(vals).mean().item())
            out[f"avg_{key}"] = avg_vals

        return out

    def training_step(self, batch, batch_idx: int):
        X, y = batch
        y_preds = self.forward(X)

        out = self._compute_pred_metrics(y_preds, y, phase="train")

        self.log_dict(
            out,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )

        return out

    def training_epoch_end(self, training_step_outputs):
        out = self._agg_stats(training_step_outputs)

        self.log_dict(
            out,
            on_step=False,
            on_epoch=True,
            logger=True,
        )

#     def validation_step(self, batch, batch_idx: int):
#         X, y = batch
#         y_preds = self.forward(X)

#         out = self._compute_pred_metrics(y_preds, y, phase="val")

#         self.log_dict(
#             out,
#             on_step=False,
#             on_epoch=True,
#             logger=True,
#         )

#         return out

#     def validation_epoch_end(self, validation_step_outputs):
#         out = self._agg_stats(validation_step_outputs)

#         self.log_dict(
#             out,
#             on_step=False,
#             on_epoch=True,
#             logger=True,
#         )

    def test_step(self, batch, batch_idx: int):
        X, y = batch
        y_preds = self.forward(X)

        out = self._compute_pred_metrics(y_preds, y, phase="test")

        self.log_dict(
            out,
            on_step=False,
            on_epoch=True,
            logger=True,
        )

        return out

    def test_epoch_end(self, test_step_outputs):
        out = self._agg_stats(test_step_outputs)

        self.log_dict(
            out,
            on_step=False,
            on_epoch=True,
            logger=True,
        )

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def train_dataloader(self):
        df_train = HFDataset(self.hfdf["train"])

        train_dataloader = torch.utils.data.DataLoader(
            dataset=df_train,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=8,
            collate_fn=self.fn_pad_sequences,
        )

        return train_dataloader

#     def val_dataloader(self):
#         df_eval = HFDataset(self.hfdf["eval"])

#         eval_dataloader = torch.utils.data.DataLoader(
#             dataset=df_eval,
#             batch_size=self.batch_size,
#             shuffle=False,
#             num_workers=8,
#             collate_fn=self.fn_pad_sequences,
#         )

#         return eval_dataloader

    def test_dataloader(self):
        df_test = HFDataset(self.hfdf["test"])

        test_dataloader = torch.utils.data.DataLoader(
            dataset=df_test,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=8,
            collate_fn=self.fn_pad_sequences,
        )

        return test_dataloader

In [6]:
def main(args):
    configs = [
#         (512, 32),
#         (256, 48),
#         (128, 64),
        (64, 128),
    ]

    for hidden_size, batch_size in configs:
        accumulate_grad_batches = 128 // batch_size

        model = LitSegmenterBaseline(
            hidden_size=hidden_size,
            batch_size=batch_size,
            tokenizer_uri="../tokenizers/6000_subwords/tokenizer.json",
            dataset_uri="./final_curated_dataset_for_training",
        )

        trainer = pl.Trainer.from_argparse_args(
            args,
            overfit_batches=0.001 if DEBUG_RUN else 0.0,
            accumulate_grad_batches=accumulate_grad_batches,
        )

        trainer.fit(model)

        if not DEBUG_RUN:
            trainer.test()


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser = pl.Trainer.add_argparse_args(parser)

    args = parser.parse_args(
        """
        --gpu 1
        --max_epochs 3
        --log_every_n_steps 1000
        --precision 32
    """.split()
    )

    main(args)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type      | Params
-----------------------------------------
0 | embeddings | Embedding | 4.6 M 
1 | lstm       | LSTM      | 5.3 M 
2 | lin_out    | Linear    | 4.1 K 
-----------------------------------------
9.9 M     Trainable params
0         Non-trainable params
9.9 M     Total params
39.453    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/media/nvme/segmentador/venvs/env3.9.10/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:207: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(
/media/nvme/segmentador/venvs/env3.9.10/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1398: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  rank_zero_warn(
Restoring states from the checkpoint path at /media/nvme/segmentador/notebooks/lightning_logs/version_3/checkpoints/epoch=2-step=3737.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model

Testing: 0it [00:00, ?it/s]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type      | Params
-----------------------------------------
0 | embeddings | Embedding | 4.6 M 
1 | lstm       | LSTM      | 2.1 M 
2 | lin_out    | Linear    | 2.1 K 
-----------------------------------------
6.7 M     Trainable params
0         Non-trainable params
6.7 M     Total params
26.845    Total estimated model params size (MB)


...

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'avg_test_cls_0_precision': 0.9983312487602234,
 'avg_test_cls_0_recall': 0.9993693232536316,
 'avg_test_cls_1_precision': 0.9855096340179443,
 'avg_test_cls_1_recall': 0.9620329737663269,
 'avg_test_cls_2_precision': 0.9168782830238342,
 'avg_test_cls_2_recall': 0.8202478289604187,
 'avg_test_cls_3_precision': 0.7135547995567322,
 'avg_test_cls_3_recall': 0.5649248957633972,
 'avg_test_loss': 0.01007597055286169,
 'avg_test_macro_f1_score': 0.8680000305175781,
 'avg_test_macro_precision': 0.9035684466362,
 'avg_test_macro_recall': 0.8366436958312988,
 'test_cls_0_precision': 0.998330295085907,
 'test_cls_0_recall': 0.999370276927948,
 'test_cls_1_precision': 0.9855092763900757,
 'test_cls_1_recall': 0.9620238542556763,
 'test_cls_2_precision': 0.9171323776245117,
 'test_cls_2_recall': 0.8202000856399536,
 'test_cls_3_precision': 0.7137227058410645,
 'test_cls_3_recall': 0.564631

Training: 0it [00:00, ?it/s]

Restoring states from the checkpoint path at /media/nvme/segmentador/notebooks/lightning_logs/version_5/checkpoints/epoch=2-step=3737.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /media/nvme/segmentador/notebooks/lightning_logs/version_5/checkpoints/epoch=2-step=3737.ckpt


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'avg_test_cls_0_precision': 0.99830561876297,
 'avg_test_cls_0_recall': 0.999434769153595,
 'avg_test_cls_1_precision': 0.9863963723182678,
 'avg_test_cls_1_recall': 0.960133969783783,
 'avg_test_cls_2_precision': 0.9214738011360168,
 'avg_test_cls_2_recall': 0.8192890286445618,
 'avg_test_cls_3_precision': 0.7188460826873779,
 'avg_test_cls_3_recall': 0.5316470861434937,
 'avg_test_loss': 0.008982779458165169,
 'avg_test_macro_f1_score': 0.8646918535232544,
 'avg_test_macro_precision': 0.9062554240226746,
 'avg_test_macro_recall': 0.8276262879371643,
 'test_cls_0_precision': 0.9983052611351013,
 'test_cls_0_recall': 0.9994353652000427,
 'test_cls_1_precision': 0.9863983988761902,
 'test_cls_1_recall': 0.9601314067840576,
 'test_cls_2_precision': 0.9216869473457336,
 'test_cls_2_recall': 0.8193143010139465,
 'test_cls_3_precision': 0.7187364101409912,
 'test_cls_3_recall': 0.5313

# Results

## 512 hidden units
{'avg_test_cls_0_precision': 0.9982461929321289,
 'avg_test_cls_0_recall': 0.9994046688079834,
 'avg_test_cls_1_precision': 0.986571729183197,
 'avg_test_cls_1_recall': 0.9582132697105408,
 'avg_test_cls_2_precision': 0.9132260084152222,
 'avg_test_cls_2_recall': 0.8215959072113037,
 'avg_test_cls_3_precision': 0.7250827550888062,
 'avg_test_cls_3_recall': 0.535641610622406,
 'avg_test_loss': 0.010228903032839298,
 'avg_test_macro_f1_score': 0.8643761277198792,
 'avg_test_macro_precision': 0.905781626701355,
 'avg_test_macro_recall': 0.8287138938903809,
 'test_cls_0_precision': 0.9982464909553528,
 'test_cls_0_recall': 0.9994055032730103,
 'test_cls_1_precision': 0.9865930676460266,
 'test_cls_1_recall': 0.9582584500312805,
 'test_cls_2_precision': 0.9132982492446899,
 'test_cls_2_recall': 0.8214690089225769,
 'test_cls_3_precision': 0.7250284552574158,
 'test_cls_3_recall': 0.5351755023002625,
 'test_loss': 0.010232209227979183,
 'test_macro_f1_score': 0.8643065094947815,
 'test_macro_precision': 0.9057917594909668,
 'test_macro_recall': 0.8285772800445557}

## 256 hidden units
{'avg_test_cls_0_precision': 0.9983312487602234,
 'avg_test_cls_0_recall': 0.9993693232536316,
 'avg_test_cls_1_precision': 0.9855096340179443,
 'avg_test_cls_1_recall': 0.9620329737663269,
 'avg_test_cls_2_precision': 0.9168782830238342,
 'avg_test_cls_2_recall': 0.8202478289604187,
 'avg_test_cls_3_precision': 0.7135547995567322,
 'avg_test_cls_3_recall': 0.5649248957633972,
 'avg_test_loss': 0.01007597055286169,
 'avg_test_macro_f1_score': 0.8680000305175781,
 'avg_test_macro_precision': 0.9035684466362,
 'avg_test_macro_recall': 0.8366436958312988,
 'test_cls_0_precision': 0.998330295085907,
 'test_cls_0_recall': 0.999370276927948,
 'test_cls_1_precision': 0.9855092763900757,
 'test_cls_1_recall': 0.9620238542556763,
 'test_cls_2_precision': 0.9171323776245117,
 'test_cls_2_recall': 0.8202000856399536,
 'test_cls_3_precision': 0.7137227058410645,
 'test_cls_3_recall': 0.5646311640739441,
 'test_loss': 0.010080181993544102,
 'test_macro_f1_score': 0.8680046200752258,
 'test_macro_precision': 0.9036736488342285,
 'test_macro_recall': 0.8365563750267029}


## 128 hidden units
{'avg_test_cls_0_precision': 0.99830561876297,
 'avg_test_cls_0_recall': 0.999434769153595,
 'avg_test_cls_1_precision': 0.9863963723182678,
 'avg_test_cls_1_recall': 0.960133969783783,
 'avg_test_cls_2_precision': 0.9214738011360168,
 'avg_test_cls_2_recall': 0.8192890286445618,
 'avg_test_cls_3_precision': 0.7188460826873779,
 'avg_test_cls_3_recall': 0.5316470861434937,
 'avg_test_loss': 0.008982779458165169,
 'avg_test_macro_f1_score': 0.8646918535232544,
 'avg_test_macro_precision': 0.9062554240226746,
 'avg_test_macro_recall': 0.8276262879371643,
 'test_cls_0_precision': 0.9983052611351013,
 'test_cls_0_recall': 0.9994353652000427,
 'test_cls_1_precision': 0.9863983988761902,
 'test_cls_1_recall': 0.9601314067840576,
 'test_cls_2_precision': 0.9216869473457336,
 'test_cls_2_recall': 0.8193143010139465,
 'test_cls_3_precision': 0.7187364101409912,
 'test_cls_3_recall': 0.5313534736633301,
 'test_loss': 0.008986305445432663,
 'test_macro_f1_score': 0.8646671175956726,
 'test_macro_precision': 0.9062817692756653,
 'test_macro_recall': 0.8275586366653442}